#  Highly Performant TensorFlow Batch Inference and Training  

For use cases involving large datasets, it may be necessary to perform highly performant batch inference using a cluster. In this notebook, we'll focus on how to use SageMaker batch transform to get inferences on a large datasets. To do this, we'll use a TensorFlow Serving model to do batch inference on a large dataset of images. We'll show how to use the new pre-processing and post-processing feature of the TensorFlow Serving container on Amazon SageMaker so that your TensorFlow model can make inferences directly on data in S3, and save post-processed inferences to S3.

The dataset we'll be using is the [“Challenge 2018/2019"] (https://github.com/cvdfoundation/open-images-dataset#download-the-open-images-challenge-28182019-test-set)” subset of the [Open Images V5 Dataset] (https://storage.googleapis.com/openimages/web/index.html). This subset consists of 100,00 images in .jpg format, for a total of 10GB. For demonstration, the [model] (https://github.com/tensorflow/models/tree/master/official/resnet#pre-trained-model) we'll be using is an image classification model based on the ResNet-50 architecture that has been trained on the ImageNet dataset, and which has been exported as a TensorFlow SavedModel.

We will use this model to predict the class that each model belongs to. We'll write a pre- and post-processing script and package the script with our TensorFlow SavedModel, and demonstrate how to get inferences on large datasets with SageMaker batch transform quickly, efficiently, and at scale, on GPU-accelerated instances.

## Setup 

We'll begin with some necessary imports, and get an Amazon SageMaker session to help perform certain tasks, as well as an IAM role with the necessary permissions.

In [25]:
%matplotlib inline
import numpy as np
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

region = sagemaker_session.boto_region_name
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-tf-batch-inference-jpeg-images-python-sdk'
print('Region: {}'.format(region))
print('S3 URI: s3://{}/{}'.format(bucket, prefix))
print('Role:   {}'.format(role))

Region: us-west-2
S3 URI: s3://sagemaker-us-west-2-038453126632/sagemaker/DEMO-tf-batch-inference-jpeg-images-python-sdk
Role:   arn:aws:iam::038453126632:role/service-role/AmazonSageMaker-ExecutionRole-20180718T141171


## Inspecting the SavedModel

In order to make inferences, we'll have to preprocess our image data in S3 to match the serving signature of our TensorFlow SavedModel (https://www.tensorflow.org/guide/saved_model), which we can inspect using the saved_model_cli (https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/saved_model_cli.py).  This is the serving signature of the ResNet-50 v2 (NCHW, JPEG) (https://github.com/tensorflow/models/tree/master/official/resnet#pre-trained-model) model:

In [44]:
!aws s3 cp s3://sagemaker-sample-data-{region}/batch-transform/open-images/model/resnet_v2_fp32_savedmodel_NCHW_jpg.tar.gz .
!tar -zxf resnet_v2_fp32_savedmodel_NCHW_jpg.tar.gz
!saved_model_cli show --dir resnet_v2_fp32_savedmodel_NCHW_jpg/1538687370/ --all

download: s3://sagemaker-sample-data-us-west-2/batch-transform/open-images/model/resnet_v2_fp32_savedmodel_NCHW_jpg.tar.gz to ./resnet_v2_fp32_savedmodel_NCHW_jpg.tar.gz

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: input_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: ArgMax:0
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1001)
        name: softmax_tensor:0
  Method name is: tensorflow/serving/predict

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: inpu

The SageMaker TensorFlow Serving Container uses the model’s SignatureDef named serving_default , which is declared when the TensorFlow SavedModel is exported. This SignatureDef says that the model accepts a string of arbitrary length as input, and responds with classes and their probabilities. With our image classification model, the input string will be a base-64 encoded string representing a JPEG image, which our SavedModel will decode.

## Writing a pre- and post-processing script

We will package up our SavedModel with a Python script named `inference.py`, which will pre-process input data going from S3 to our TensorFlow Serving model, and post-process output data before it is saved back to S3:

In [45]:
!pygmentize code/inference.py

# Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

import base64
import io
import json
import requests
import tensorflow as tf
from google.protobuf.json_format import MessageToDict
from string import whitespace

def input_handler(data, context):
    """ Pre-process request input before it is sent to TensorFlow Serving REST API

    Args:
        data (obj): the request data stream
        context (Context): an object containing request and configuration details

    Returns:
     

The input_handler intercepts inference requests, base-64 encodes the request body, and formats the request body to conform to TensorFlow Serving’s REST API (https://www.tensorflow.org/tfx/serving/api_rest). The return value of the input_handler function is used as the request body in the TensorFlow Serving request. Binary data must use key "b64", according to the TFS REST API (https://www.tensorflow.org/tfx/serving/api_rest#encoding_binary_values), and since our serving signature’s input tensor has the suffix "\_bytes", the encoded image data under key "b64" will be passed to the "image\_bytes" tensor. Some serving signatures may accept a tensor of floats or integers instead of a base-64 encoded string, but for binary data (including image data), it is recommended that your SavedModel accept a base-64 encoded string for binary data, since JSON representations of binary data can be large.

Each incoming request originally contains a serialized JPEG image in its request body, and after passing through the input_handler, the request body contains the following, which our TensorFlow Serving accepts for inference:

`{"instances": [{"b64":"[base-64 encoded JPEG image]"}]}`

The first field in the return value of `output_handler` is what SageMaker Batch Transform will save to S3 as this example’s prediction. In this case, our `output_handler` passes the content on to S3 unmodified.

Pre- and post-processing functions let you perform inference with TensorFlow Serving on any data format, not just images. To learn more about the `input_handler` and `output_handler`, consult the SageMaker TensorFlow Serving Container README (https://github.com/aws/sagemaker-tensorflow-serving-container/blob/master/README.md).

## Packaging a Model

After writing a pre- and post-processing script, you’ll need to package your TensorFlow SavedModel along with your script into a `model.tar.gz` file, which we’ll upload to S3 for the SageMaker TensorFlow Serving Container to use. Let's package the SavedModel with the `inference.py` script and examine the expected format of the `model.tar.gz` file:

In [49]:
!tar -cvzf model.tar.gz code --directory=resnet_v2_fp32_savedmodel_NCHW_jpg 1538687370

code/
code/.ipynb_checkpoints/
code/inference.py
1538687370/
1538687370/saved_model.pb
1538687370/variables/
1538687370/variables/variables.data-00000-of-00001
1538687370/variables/variables.index


`1538687370` refers to the model version number of the SavedModel, and this directory contains our SavedModel artifacts. The code directory contains our pre- and post-processing script, which must be named `inference.py`. I can also include an optional `requirements.txt` file, which is used to install dependencies with `pip` from the Python Package Index before the Transform Job starts, but we don’t need any additional dependencies in this case, so we don't include a requirements file.

We will use this `model.tar.gz` when we create a SageMaker Model, which we will use to run Transform Jobs. To learn more about packaging a model, you can consult the SageMaker TensorFlow Serving Container README (https://github.com/aws/sagemaker-tensorflow-serving-container/blob/master/README.md).

## Run a Batch Transform job

Next, we'll run a Batch Transform job using our data processing script and GPU-based Amazon SageMaker Model. More specifically, we'll perform inference on a cluster of two instances, though we can choose more or fewer. The objects in the S3 path will be distributed between the instances. In this example, our input can't be split by newline characters and batched together, so the cluster will receive one HTTP request per serialized image.

The code below creates a SageMaker Model entity that will be used for Batch inference, and runs a Transform Job using that Model. The Model contains a reference to the TFS container, and the `model.tar.gz` containing our TensorFlow SavedModel and the pre- and post-processing `inference.py` script.

In [50]:
import os
import sagemaker
from sagemaker.tensorflow.serving import Model

s3_path = 's3://{}/{}'.format(bucket, prefix)

model_data = sagemaker_session.upload_data('model.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model'))
                                           
tensorflow_serving_model = Model(model_data=model_data,
                                 role=role,
                                 framework_version='1.13',
                                 sagemaker_session=sagemaker_session)

input_path = 's3://sagemaker-sample-data-{}/batch-transform/open-images/jpg'.format(region)

Before we create a Transform Job, let's inspect some of our input data. Here's an example, the first image in our dataset:



<img src="sample_image/00000b4dcff7f799.jpg">

The data in the input path consists of 100,000 JPEG images of varying sizes and shapes. Here is a subset:

In [56]:
!echo "Transform input path: {input_path}"
!aws s3 ls {input_path}/000 --human-readable

Transform input path: s3://sagemaker-sample-data-us-west-2/batch-transform/open-images/jpg
2019-07-09 22:19:18  126.2 KiB 00000b4dcff7f799.jpg
2019-07-09 22:19:18  115.8 KiB 00001a21632de752.jpg
2019-07-09 22:19:18  151.0 KiB 0000d67245642c5f.jpg
2019-07-09 22:19:18  159.9 KiB 0001244aa8ed3099.jpg
2019-07-09 22:19:18  115.0 KiB 000172d1dd1adce0.jpg
2019-07-09 22:19:18   65.4 KiB 0001c8fbfb30d3a6.jpg
2019-07-09 22:19:18   70.4 KiB 0001dd930912683d.jpg
2019-07-09 22:19:18   73.0 KiB 0002c96937fae3b3.jpg
2019-07-09 22:19:18  109.2 KiB 0002f94fe2d2eb9f.jpg
2019-07-09 22:19:18  119.2 KiB 000305ba209270dc.jpg
2019-07-09 22:19:18  119.5 KiB 000313fed9979d24.jpg
2019-07-09 22:19:18   77.2 KiB 0003a523fa9b2a3f.jpg
2019-07-09 22:19:18   84.9 KiB 0003d1c3be9ed3d6.jpg
2019-07-09 22:19:18   82.9 KiB 000455be7b222c04.jpg
2019-07-09 22:19:18  104.8 KiB 0004fdbc5b94c7c2.jpg
2019-07-09 22:19:18  144.0 KiB 0005339c44e6071b.jpg
2019-07-09 22:19:19   75.2 KiB 0005aea8c9144c77.jpg
2019-07-09 22:19:19   71.

Now that we’ve created a SageMaker Model, we can use it to run batch predictions using Batch Transform. We specify the input S3 data, content type of the input data, the output S3 data, and instance type and count.

For improved performance, we specify two additional parameters `max_concurrent_transforms` and `max_payload`, which control the maximum number of parallel requests that can be sent to each instance in a transform job at a time, and the maximum size of each request body.

When performing inference on entire S3 objects that cannot be split by newline characters, such as images, it is recommended that you set `max_payload` to be slightly larger than the largest S3 object in your dataset, and that you experiment with the `max_concurrent_transforms` parameter in powers of two to find a value that maximizes throughput for your model. For example, we’ve set `max_concurrent_transforms` to 64 after experimenting with powers of two, and we set `max_payload` to 1, since the largest object in our S3 input is less than one megabyte.

In [ ]:
output_path = os.path.join(s3_path, 'output')
tensorflow_serving_transformer = tensorflow_serving_model.transformer(
                                     instance_count=2,
                                     instance_type='ml.p2.xlarge',
                                     max_concurrent_transforms=64,
                                     max_payload=1,
                                     output_path=output_path)

print('Transform input S3 path:  {}'.format(input_path))
print('Transform output S3 path: {}'.format(output_path))
tensorflow_serving_transformer.transform(input_path, content_type='application/x-image')
tensorflow_serving_transformer.wait()

Transform input S3 path:  s3://sagemaker-sample-data-us-west-2/batch-transform/open-images/jpg
Transform output S3 path: s3://sagemaker-us-west-2-038453126632/sagemaker/DEMO-tf-batch-inference-jpeg-images-python-sdk/output
..........................................................................................................................................................................................................................

After our transform job finishes, we find one S3 object in the output path for each object in the input path. This object contains the inferences from our model for that object, and has the same name as the corresponding input object, but with `.out` appended to it.

In [ ]:
!aws s3 ls {output_path}/000 --human-readable

Inspecting one of the output objects, we find the prediction from our TensorFlow Serving model. This is from the example image displayed above:

In [ ]:
!aws s3 cp {output_path}/00000b4dcff7f799.jpg.out .
!cat 00000b4dcff7f799.jpg.out

In [18]:
from sagemaker.tensorflow.serving import Model

tensorflow_serving_model = Model(source_dir='code',
                                 entry_point='code/inference.py',
                                 model_data=estimator_dist.model_data,
                                 role=role,
                                 framework_version='1.12',
                                 sagemaker_session=sagemaker_session)

input_data_path='s3://sagemaker-sample-data-{}/tensorflow/cifar10/images/png'.format(sagemaker_session.boto_region_name)
output_data_path='s3://{}/{}/{}'.format(bucket, prefix, 'batch-predictions')
batch_instance_count=2
batch_instance_type = 'ml.p2.xlarge'
concurrency=32
max_payload_in_mb=1

transformer = tensorflow_serving_model.transformer(
    instance_count=batch_instance_count,
    instance_type=batch_instance_type,
    max_concurrent_transforms=concurrency,
    max_payload=max_payload_in_mb,
    output_path=output_data_path
)

transformer.transform(data=input_data_path, content_type='application/x-image')
transformer.wait()

.........................................................!


### Inspect Batch Transform output

Finally, we can inspect the output files of our Batch Transform job to see the predictions.  First we'll download the prediction files locally, then extract the predictions from them.

In [19]:
!aws s3 cp --quiet --recursive $transformer.output_path ./batch_predictions

^C


In [ ]:
import json
import re

total = 0
correct = 0
predicted = []
actual = []
labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
for entry in os.scandir('batch_predictions'):
    try:
        if entry.is_file() and entry.name.endswith("out"):
            with open(entry, 'r') as f:
                jstr = json.load(f)
                results = [float('%.3f'%(item)) for sublist in jstr['predictions'] for item in sublist]
                class_index = np.argmax(np.array(results))
                predicted_label = labels[class_index]
                predicted.append(predicted_label)
                actual_label = re.search('([a-zA-Z]+).png.out', entry.name).group(1)
                actual.append(actual_label)
                is_correct = (predicted_label in entry.name) or False
                if is_correct:
                    correct += 1
                total += 1
    except Exception as e:
        print(e)
        continue

Let's calculate the accuracy of the predictions.  

In [ ]:
print('Out of {} total images, accurate predictions were returned for {}'.format(total, correct))
accuracy = correct / total
print('Accuracy is {:.1%}'.format(accuracy))

The accuracy from the batch transform job on 10000 test images never seen during training is fairly close to the accuracy achieved during training on the validation set.  This is an indication that the model is not overfitting and should generalize fairly well to other unseen data. 

Next we'll plot a confusion matrix, which is a tool for visualizing the performance of a multiclass model. It has entries for all possible combinations of correct and incorrect predictions, and shows how often each one was made by our model. Ours will be row-normalized: each row sums to one, so that entries along the diagonal correspond to recall. 

In [ ]:
import pandas as pd
import seaborn as sns

confusion_matrix = pd.crosstab(pd.Series(actual), pd.Series(predicted), rownames=['Actuals'], colnames=['Predictions'], normalize='index')
sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix')  

If our model had 100% accuracy, and therefore 100% recall in every class, then all of the predictions would fall along the diagonal of the confusion matrix.  Here our model definitely is not 100% accurate, but manages to achieve good recall for most of the classes, though it performs worse for some classes, such as cats.  

## Model Deployment with Amazon Elastic Inference

Amazon SageMaker also lets you deploy a TensorFlow Serving model to a hosted Endpoint for real-time inference. As we will see, the processes for setting up hosted endpoints and Batch Transform jobs have significant differences.  Additionally, we will discuss why and how to use Amazon Elastic Inference with the hosted endpoint.

### Deploying the Model

When considering the overall cost of a machine learning workload, inference often is the largest part, up to 90% of the total.  If a GPU instance type is used for real time inference, it typically is not fully utilized because, unlike training, real time inference does not involve continuously inputting large batches of data to the model.  Elastic Inference provides GPU acceleration suited for inference, allowing you to add inference acceleration to a hosted endpoint for a fraction of the cost of using a full GPU instance.

Instead of a Transformer object, we'll instantiate a Predictor object now. The `deploy` method of the Estimator object instantiates a Predictor object representing an endpoint which serves prediction requests in near real time.  To utilize Elastic Inference with the SageMaker TFS container, simply provide an `accelerator_type` parameter, which determines the type of accelerator that is attached to your endpoint. Refer to the **Inference Acceleration** section of the [instance types chart](https://aws.amazon.com/sagemaker/pricing/instance-types) for a listing of the supported types of accelerators. 

Here we'll use a general purpose CPU compute instance type along with an Elastic Inference accelerator:  together they are much cheaper than the smallest P3 GPU instance type.

In [ ]:
predictor = estimator_dist.deploy(initial_instance_count=1,
                                  instance_type='ml.m5.xlarge',
                                  accelerator_type='ml.eia1.medium')

###  Real time inference
  
Now that we have a Predictor object wrapping a real time Amazon SageMaker hosted endpoint, we'll define the label names and look at a sample of 10 images, one from each class.

In [ ]:
from IPython.display import Image, display

images = []
for entry in os.scandir('sample-img'):
    if entry.is_file() and entry.name.endswith("png"):
        images.append('sample-img/' + entry.name)

for image in images:
    display(Image(image))

Next, we'll modify some properties of the Predictor object created by the `deploy` method call above.  The TFS container in Amazon SageMaker by default uses the TFS REST API, which requires requests in a specific JSON format.  However, for many real time use cases involving image data it is more convenient to have the client application send the image data directly to an endpoint for predictions, without converting and preprocessing it on the client side. 

Fortunately, our endpoint includes the same pre/post-processing script used in the Batch Transform section of this notebook because the same model artifact is used in both cases. This model artifact includes the same `inference.py` script.  With this preprocessing script in place, we just specify the Predictor's content type as `application/x-image` and override the default serializer. Then we can simply provide the raw .png image bytes to the Predictor.  

In [ ]:
predictor.content_type = 'application/x-image'
predictor.serializer = None

labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

def get_prediction(file_path):
    
    with open(file_path, "rb") as image:
        f = image.read()
    b = bytearray(f)
    return labels[np.argmax(predictor.predict(b)['predictions'], axis=1)[0]]

In [ ]:
predictions = [get_prediction(image) for image in images]
print(predictions)

# Extensions

Although we did not demonstrate them in this notebook, Amazon SageMaker provides additional ways to make distributed training more efficient for very large datasets:
- **VPC training**:  performing Horovod training inside a VPC improves the network latency between nodes, leading to higher performance and stability of Horovod training jobs.

- **Pipe Mode**:  using [Pipe Mode](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html#your-algorithms-training-algo-running-container-inputdataconfig) reduces startup and training times.  Pipe Mode streams training data from S3 as a Linux FIFO directly to the algorithm, without saving to disk.  For a small dataset such as CIFAR-10, Pipe Mode does not provide any advantage, but for very large datasets where training is I/O bound rather than CPU/GPU bound, Pipe Mode can substantially reduce startup and training times.

# Cleanup

To avoid incurring charges due to a stray endpoint, delete the Amazon SageMaker endpoint if you no longer need it:

In [ ]:
sagemaker_session.delete_endpoint(predictor.endpoint)